In [1]:
import sys

# Increase recursion limit to prevent potential issues
sys.setrecursionlimit(100000)

The sys.setrecursionlimit function is used to increase the recursion limit, which helps prevent potential recursion errors when running complex models with deep nested functions or when using certain libraries like TensorFlow.

- **`keras_tuner`**: Used for hyperparameter tuning.
- **`Sequential`**: A linear stack of layers in Keras.
- **`Dense`**, **`Flatten`**: Common Keras layers.
- **`mnist`**: The MNIST dataset, a standard dataset for image classification.
- **`Adam`**: An optimizer in Keras.

In [2]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
import os
import warnings

# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all logs, 1 = filter out INFO, 2 = filter out INFO and WARNING, 3 = ERROR only

- **`mnist.load_data()`**: Loads the dataset, returning training and validation splits.
- **`x_train / 255.0`**: Normalizes the pixel values to be between 0 and 1.
- **`print(f'...')`**: Displays the shapes of the training and validation datasets.

In [ ]:
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0

print(f'Training data shape: {x_train.shape}')
print(f'Validation data shape: {x_val.shape}')

### Define the model with Hyperparameters
**Define a model-building function:**
- Create a function `build_model` that takes a `HyperParameters` object as input.
- Use the `HyperParameters` object to define the number of units in a dense layer and the learning rate for the optimizer.
- Compile the model with sparse categorical cross-entropy loss and Adam optimizer.


- **`hp.Int('units', ...)`**: Defines the number of units in the Dense layer as a hyperparameter.
- **`hp.Float('learning_rate', ...)`**: Defines the learning rate as a hyperparameter.
- **`model.compile()`**: Compiles the model with the Adam optimizer and sparse categorical cross-entropy loss.


In [3]:
# Define a model-building function 

def build_model(hp):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model
